## Bayes learn (chapter 5)

### 5.1 Odds ratio

通常用来表示概率的是一个[0,1]之间的数字，但这并不是唯一一个方式。还有一个常用方式是`odds`，又称为优势比 (`odds in favor`)。一个事件的优势比是指它发生的概率与不发生的概率的比值。假如我认为一个球队在一场比赛中有75%的概率会赢，那么这支球队赢球的优势比就是`3:1`。

当我们期望的概率比较低时，我们也会用劣势比(`odds against`)。比如我认为自己赌马失败的概率为10%，那么odds就是`9:1`。

贝叶斯定理是： $p(H|D) = \frac{p(H)p(D|H)}{p(D)}$

假如有2个假设A和B，则它们的后验概率比值为：$\frac{p(A|D)}{p(B|D)}=\frac{p(A)p(D|A)}{p(B)p(D|B)}$

我们用odds重写这个公式，令o(A)为支持A的优势比，则公式为$o(A|D)=o(A)\frac{p(D|A)}{p(D|B)}$。o(A)为先验概率优势比p(A)/p(B)，o(A|D)为后验概率优势比。

也就是说，后验优势比是先验优势比乘以似然值。这对于我们计算贝叶斯公式会有很大简化作用。

*我们考虑之前的糖果问题：假如有两盒糖果。第一盒中有30个香草味和10个巧克力味；第二盒中香草味和巧克力味各有20个。现在我们随机选择了一个盒子，又从中选择了一个糖果，结果发现为巧克力味，那么这个糖果来源于盒1的概率是多少？*

在这个问题中先验odds是1:1，似然值为$\frac{3}{4}/\frac{1}{2}$，也就是3/2，所以后验odds也是3:2，则糖果来源于罐1的概率就是3/5。

*两个人在杀人现场留下血迹，一个为O型血（人群中出现概率为60%），一个为AB型血（人群中出现概率为1%）。其中一个嫌疑人测试为O型血，跟定这些证据的情况下判断是否其中一个血迹来源于这个嫌疑人？*

为了回答这个问题，我们需要定义什么是观察到的数据支持假设，什么是观察到的数据不支持假设。直觉上，我们认为在观察到这个数据后假设发生的概率更大时数据就支持假设。

在糖果问题中，先验优势比似1:1，后验优势比是3:2，所以我们说`香草味的糖果`这个证据支持`罐1`假设。

Bayes's定理的优势比形式给这个直觉一个更准确的定义： $\frac{o(A|D)}{o(A)} = \frac{p(D|A)}{p(D|B)} $。

等式左侧为为后验优势比与先验优势比的比值，等式右侧为似然值的比值，也称为贝叶斯因子 (Bayes factor)。如果贝叶斯因子值大于1，就说明数据更有可能支持A假设。此时后验优势比与先验优势比的比值也大于1，也就是说在数据存在的情况下，支持A的优势比升高了；如果贝叶斯因子小于1，则说明数据更可能支持B假设，支持A的优势比降低了。

那么我们回到`Oliver的血`的问题。如果Oliver是在现场留下血迹的人中的一个，那么他留下的是O型血，另外一个人留下AB型血的概率是1%。

如果现场中的血不来自于Oliver，就应该来自于随机的另外两个人，其中一个为O型血，一个为AB型血，或者反过来，因此概率为2 * 0.6 * 0.01 = 1.2%。

似然值的比值为1/1.2，因此单单验血数据其实不支持Oliver为嫌疑人之一。

这个例子有一些反直觉，就是说与假设一致的数据模式却不一定支持假设。

这个问题其实是似然值的问题，判断这个证据对确定嫌疑人的贡献有多大。如果这个血迹来源于这个人，似然值为1 * 1%=1%，如果这个血迹不来源于这个人，则似然值为2 * 60% * 1% = 1.2%。

### 5.4 Addends （加数）

Bayesian统计学的基本操作是利用观察到的数据计算似然值更新先验概率得到后验概率分布。

但是在解决实际问题时，还会引入其它的操作，包括倍数(scaling)、加和(addition)和其它数学操作，最大值、最小值和它们的混合运算。

还是用“地下城和勇士”的例子，在这个角色扮演游戏中，玩家的决定通过掷骰子来决定。在游戏开始时，玩家通过抛掷3个六面的骰子对获得的数字加和得到它们扮演的角色的力量值、智力值、智慧值、敏捷度、组织能力和领导力。

那么我们好奇这一数字加和的分布是什么？ 有两个方法可以计算：

* 模拟：给定一个概率密度函数代表一个骰子的结果分布，然后随机采样，加和，计算模拟加和的累计分布
* 穷举：给定2个概率密度函数，穷举所有可能的数值对，计算其总和的概率分布



In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

def lineplot(x, y):
    plt.plot(x,y)
    plt.xlabel('Hypotheses')
    plt.ylabel('Probabilities')

In [2]:
from __future__ import division, unicode_literals
import logging

class Bayes(object):
    """A bayes class, mainly a dictionary"""
    def __init__(self, hypos=None, name=''):
        """
        Initialize the distribution.
        
        hypos: sequence of hypotheses
        """
        self.name = name
        self.pmf = {}
        if hypos is None:
            return
        
        self.hypos = hypos
        # Initiate the class object
        # Three initalize methods are used to deal with different types of input
        # 
        init_methods = [
            self.InitPmf,
            self.InitMapping,  #A dict
            self.InitSequence, #equal probability for all hypos
            self.InitFailure,
        ]
        
        for method in init_methods:
            try:
                method(hypos)
                break
            except AttributeError:
                continue
        
        if len(self):
            self.Normalize()
    
    def __str__(self):
        '''
        Stringlize self.pmf
        '''
        tmpL = ["Probability table\n"]
        for hypo, prob in sorted(self.pmf.iteritems()):
            tmpL.append('\t'.join(['',str(hypo), str(prob)]))
        return '\n'.join(tmpL)
    
    def InitSequence(self, hypos):
        """
        Initialize with a sequence of hypos with equal probabilities.
        
        hypos: ['H1','H2','H3',...]
        """
        for hypo in hypos:
            self.Set(hypo, 1)
    
    def InitMapping(self, hypos):
        """
        Initialize with a map from value to probablity (a dict).
        
        hypos = {'H1':1, 'H2':5, 'H3':4}
        """
        for hypo, prob in hypos.iteritems():
            self.Set(hypo, prob)
    
    def InitPmf(self, hypos):
        """
        Initialize with a Bayes object.
        
        hypos = Bayes()
        """
        for hypo, prob in hypos.iteritems():
            self.Set(hypo, prob)
    
    def InitFailure(self, hypos):
        """Raise an errot"""
        raise ValueError("None of the initialization methods works.")
    
    def __len__(self):
        return len(self.pmf)
    
    def Set(self, hypo, prob=0):
        """
        Set hypo-prob pair
        """
        self.pmf[hypo] = prob
    
    def Print(self):
        """Print the values and freqs in asending order."""
        for hypo, prob in sorted(self.pmf.iteritems()):
            print hypo, prob
    
    def Normalize(self):
        """
        Normalize probability
        """
        total = float(sum(self.pmf.values()))
        if total == 0.0:
            raise ValueError('total probability is zero.')
            logging.warning('Normalize: total probability is zero.')
            return total
        
        factor = 1 / total
        
        for hypo in self.pmf:
            self.pmf[hypo] *= factor
    
    def Items(self):
        '''Return two lists, hypos_list and probability_list'''
        if isinstance(self.hypos, list):
            hypos = self.hypos
        else:
            hypos = self.pmf.keys()
            hypos.sort()
        probs = [self.pmf[hypo] for hypo in hypos]
        return hypos, probs
    
    def Max(self):
        '''Return the hypothesis with maximum posterior probability'''
        max_prob, max_hypo = max([(prob,hypo) for hypo, prob in self.pmf.iteritems()])
        return max_hypo, max_prob
    
    def Mult(self, hypo, likelihood):
        '''
        Update given hypo probability by given likelihood
        '''
        self.pmf[hypo] = self.pmf.get(hypo,0) * likelihood
    
    def Prob(self, hypo, default=0):
        """
        Get the probability of given hypo.
        """
        return self.pmf.get(hypo, default)
    
    def Update(self,dataL):
        '''
        Update all hypo probability by given obervation.
        
        dataL: A list of observations.
        '''
        for data in dataL:
            for hypo, prob in self.pmf.iteritems():
                self.pmf[hypo] = prob * self.Likelihood(hypo, data)
        self.Normalize()
    
    def Likelihood(self, hypo, data):
        '''
        Re-constructed in child class
        '''
        psss
    
    def Mean(self):
        '''
        Compute the weighted hypothesis using posterior probabilities.
        '''
        total = 0
        for hypo, prob in self.pmf.iteritems():
            total += hypo * prob
        return total
    
    def Percentile(self, percentage):
        '''
        Compute a percentile for a given percentage.
        
        percentage: float from 0 to 100.
        ''' 
        if isinstance(self.hypos, list):
            hypos = self.hypos
        else:
            hypos = self.pmf.keys()
            hypos.sort()
        total = 0
        assert 0<=percentage<=100, "percentage must between [0,100]"
        percentile_value = percentage / 100.0
        for hypo in hypos:
            total += self.pmf[hypo]
            if total >= percentile_value:
                return hypo
    def probPlot(self):
        hypos, probs = self.Items()
        plt.plot(hypos, probs)
        plt.xlabel('Hypothses')
        plt.ylabel('Probabilities')